In [1]:
# 导入模块
import os # 用于操作系统的交互，例如获取环境变量。
import openai # OpenAI 的官方库，用于与 OpenAI 服务进行交互。
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

openai.api_key  = os.environ['OPENAI_API_KEY'] # 从环境变量中获取 OPENAI_API_KEY 作为 API 密钥。
openai.api_type = "azure"
openai.api_base = "https://zjx.openai.azure.com/"
openai.api_version = "2023-05-15"

In [2]:
# 定义了一个名为 get_completion 的函数，其主要目的是使用 OpenAI 的 API 为给定的 prompt 获取模型的回复
def get_completion(prompt):
    # messages 是一个列表，其中包含一个字典，这个字典定义了一个用户角色和他们的消息内容。
    # 这是 OpenAI 的 Chat API 接受的格式，它允许多次交互（例如，先由用户发送消息，然后由模型回复，然后再由用户发送消息，等等）
    messages = [{"role": "user", "content": prompt}] 
    # 使用 try 和 except 来尝试执行某些代码，并在出现异常时捕获该异常。
    try:
        response = openai.ChatCompletion.create(
            engine="gpt35",  # 指定使用哪个模型。在这里，它是 "gpt35"。
            messages=messages,
            temperature=0, # 控制输出随机性的参数。值为 0 表示模型会产生最确定性的输出；较高的值（例如 1.0）会产生更多的随机性。
        )
        return response.choices[0].message["content"] # 从模型的响应中获取其消息内容并返回。
    except Exception as e:
        print(f"Error: {e}")
        return None

In [3]:
## 在这里，max_tokens主要限制的是输出的令牌数量，也就是模型生成的内容的最大长度。
def get_completion_from_messages(messages, 
                                 max_tokens=500):
    try:
        response = openai.ChatCompletion.create(
            max_tokens=max_tokens,
            engine="gpt35",
            messages=messages,
            temperature=0,
        )
    except Exception as e:
        print(f"Error: {e}")
    return response.choices[0].message["content"]

In [56]:
delimiter = "####"
system_message = f"""
您将接收到客户服务的查询。每条客户查询都会使用{delimiter}字符进行分隔。
请输出一个Python对象列表，每个对象都有以下格式：
    'category': <以下类别之一：计算机和笔记本、手机和配件、电视和家庭影院系统、游戏机和配件、音响设备、摄像机和摄像机>,
或者
    'products': <必须在以下允许的产品列表中查找到的产品名列表>

类别和产品必须在客户查询中找到。如果提到了某个产品，它必须与允许的产品列表中的正确类别对应。如果没有找到产品或类别，输出一个空列表。

允许的产品：

计算机和笔记本类别：
TechPro Ultrabook
BlueWave Gaming Laptop
PowerLite Convertible
TechPro Desktop
BlueWave Chromebook

手机和配件类别：
SmartX ProPhone
MobiTech PowerCase
SmartX MiniPhone
MobiTech Wireless Charger
SmartX EarBuds


电视和家庭影院系统类别：
CineView 4K TV
SoundMax Home Theater
CineView 8K TV
SoundMax Soundbar
CineView OLED TV

游戏机和配件类别：
GameSphere X
ProGamer Controller
GameSphere Y
ProGamer Racing Wheel
GameSphere VR Headset

音频设备类别：
AudioPhonic Noise-Canceling Headphones
WaveSound Bluetooth Speaker
AudioPhonic True Wireless Earbuds
WaveSound Soundbar
AudioPhonic Turntable

摄像机和摄像机类别：
FotoSnap DSLR Camera
ActionCam 4K
FotoSnap Mirrorless Camera
ZoomMaster Camcorder
FotoSnap Instant Camera

仅输出对象列表，不包括其他任何内容。
"""

user_message_1 = f"""
告诉我关于 SmartX ProPhone 和 \
FotoSnap 相机，那个 DSLR 的。\
还有告诉我关于你们的电视。
"""

messages =  [  
{'role':'system', 
 'content': system_message},    
{'role':'user', 
 'content': f"{delimiter}{user_message_1}{delimiter}"},  
] 
category_and_product_response_1 = get_completion_from_messages(messages)
print(category_and_product_response_1)


[{'category': '手机和配件'}, {'products': ['SmartX ProPhone']}, {'category': '摄像机和摄像机'}, {'products': ['FotoSnap DSLR Camera']}, {'category': '电视和家庭影院系统'}]


In [89]:
def find_category_and_product_only(user_input):
    delimiter = "####"
    system_message = f"""
您将接收到客户服务的查询。每条客户查询都会使用{delimiter}字符进行分隔。
请输出一个Python对象列表，每个对象都有以下格式：
    'category': <以下类别之一：计算机和笔记本、手机和配件、电视和家庭影院系统、游戏机和配件、音响设备、摄像机和摄像机>,
或者
    'products': <必须在以下允许的产品列表中查找到的产品名列表>

类别和产品必须在客户查询中找到。如果提到了某个产品，它必须与允许的产品列表中的正确类别对应。如果没有找到产品或类别，输出一个空列表。

允许的产品：

计算机和笔记本类别：
TechPro Ultrabook
BlueWave Gaming Laptop
PowerLite Convertible
TechPro Desktop
BlueWave Chromebook

手机和配件类别：
SmartX ProPhone
MobiTech PowerCase
SmartX MiniPhone
MobiTech Wireless Charger
SmartX EarBuds


电视和家庭影院系统类别：
CineView 4K TV
SoundMax Home Theater
CineView 8K TV
SoundMax Soundbar
CineView OLED TV

游戏机和配件类别：
GameSphere X
ProGamer Controller
GameSphere Y
ProGamer Racing Wheel
GameSphere VR Headset

音频设备类别：
AudioPhonic Noise-Canceling Headphones
WaveSound Bluetooth Speaker
AudioPhonic True Wireless Earbuds
WaveSound Soundbar
AudioPhonic Turntable

摄像机和摄像机类别：
FotoSnap DSLR Camera
ActionCam 4K
FotoSnap Mirrorless Camera
ZoomMaster Camcorder
FotoSnap Instant Camera

仅输出对象列表，不包括其他任何内容。
"""

    user_message_1 = f"""
    告诉我关于 SmartX ProPhone 和 \
    FotoSnap 相机，那个 DSLR 的。\
    还有告诉我关于你们的电视。
    """

    messages =  [  
    {'role':'system', 
     'content': system_message},    
    {'role':'user', 
     'content': f"{delimiter}{user_input}{delimiter}"},  
    ] 
    category_and_product_response_1 = get_completion_from_messages(messages)
    return category_and_product_response_1


In [90]:
user_input = "告诉我关于smartx pro手机和fotosnap相机，那个DSLR的。还有，告诉我关于你们的电视。"
response= find_category_and_product_only(user_input)
print(response)

[{'category': '手机和配件'}, {'products': ['SmartX ProPhone', 'FotoSnap DSLR Camera']}, {'category': '电视和家庭影院系统'}]


In [57]:
user_message_2 = f"""
我的路由器坏了"""
messages =  [  
{'role':'system',
 'content': system_message},    
{'role':'user',
 'content': f"{delimiter}{user_message_2}{delimiter}"},  
] 
response = get_completion_from_messages(messages)
print(response)

[]


In [58]:

products = {
    "TechPro Ultrabook": {
        "名称": "TechPro Ultrabook",
        "分类": "电脑与笔记本",
        "品牌": "TechPro",
        "型号": "TP-UB100",
        "保修期": "1 年",
        "评分": 4.5,
        "特点": ["13.3 英寸显示屏", "8GB RAM", "256GB SSD", "Intel Core i5 处理器"],
        "描述": "轻薄的日常使用超级本。",
        "价格": 799.99
    },
    "BlueWave Gaming Laptop": {
        "名称": "BlueWave Gaming Laptop",
        "分类": "电脑与笔记本",
        "品牌": "BlueWave",
        "型号": "BW-GL200",
        "保修期": "2 年",
        "评分": 4.7,
        "特点": ["15.6 英寸显示屏", "16GB RAM", "512GB SSD", "NVIDIA GeForce RTX 3060"],
        "描述": "为沉浸式体验提供的高性能游戏笔记本。",
        "价格": 1199.99
    },
    "PowerLite Convertible": {
        "名称": "PowerLite Convertible",
        "分类": "电脑与笔记本",
        "品牌": "PowerLite",
        "型号": "PL-CV300",
        "保修期": "1 年",
        "评分": 4.3,
        "特点": ["14 英寸触摸屏", "8GB RAM", "256GB SSD", "360 度铰链"],
        "描述": "具有响应式触摸屏的多功能可转换笔记本。",
        "价格": 699.99
    },
    "TechPro Desktop": {
        "名称": "TechPro 桌面电脑",
        "分类": "电脑和笔记本",
        "品牌": "TechPro",
        "型号": "TP-DT500",
        "保修期": "1年",
        "评分": 4.4,
        "特点": ["Intel Core i7 处理器", "16GB RAM", "1TB HDD", "NVIDIA GeForce GTX 1660"],
        "描述": "适用于工作和娱乐的强大桌面电脑。",
        "价格": 999.99
    },
    "BlueWave Chromebook": {
        "名称": "BlueWave Chromebook",
        "分类": "电脑和笔记本",
        "品牌": "BlueWave",
        "型号": "BW-CB100",
        "保修期": "1年",
        "评分": 4.1,
        "特点": ["11.6英寸显示屏", "4GB RAM", "32GB eMMC", "Chrome OS"],
        "描述": "用于日常任务的紧凑且经济实惠的Chromebook。",
        "价格": 249.99
    },
    "SmartX ProPhone": {
        "名称": "SmartX ProPhone",
        "分类": "智能手机和配件",
        "品牌": "SmartX",
        "型号": "SX-PP10",
        "保修期": "1年",
        "评分": 4.6,
        "特点": ["6.1英寸显示屏", "128GB 存储", "12MP 双摄像头", "5G"],
        "描述": "具有先进摄像头功能的强大智能手机。",
        "价格": 899.99
    },
    "MobiTech PowerCase": {
        "名称": "MobiTech PowerCase",
        "分类": "智能手机和配件",
        "品牌": "MobiTech",
        "型号": "MT-PC20",
        "保修期": "1年",
        "评分": 4.3,
        "特点": ["5000mAh 电池", "无线充电", "兼容 SmartX ProPhone"],
        "描述": "带有内置电池的保护壳，用于延长使用时间。",
        "价格": 59.99
    },
    "SmartX MiniPhone": {
        "名称": "SmartX MiniPhone",
        "分类": "智能手机和配件",
        "品牌": "SmartX",
        "型号": "SX-MP5",
        "保修期": "1年",
        "评分": 4.2,
        "特点": ["4.7英寸显示屏", "64GB 存储", "8MP 相机", "4G"],
        "描述": "用于基本任务的紧凑且经济实惠的智能手机。",
        "价格": 399.99
    },
    "MobiTech Wireless Charger": {
        "名字": "MobiTech Wireless Charger",
        "分类": "智能手机及配件",
        "品牌": "MobiTech",
        "型号": "MT-WC10",
        "保修": "1 年",
        "评分": 4.5,
        "特点": ["10W 快速充电", "支持Qi标准", "LED指示灯", "紧凑设计"],
        "描述": "一个便捷的无线充电器，为您提供整洁的工作空间。",
        "价格": 29.99
    },
    "SmartX EarBuds": {
        "名字": "SmartX EarBuds",
        "分类": "智能手机及配件",
        "品牌": "SmartX",
        "型号": "SX-EB20",
        "保修": "1 年",
        "评分": 4.4,
        "特点": ["真正的无线", "Bluetooth 5.0", "触摸控制", "24小时电池续航"],
        "描述": "用这款舒适的耳塞体验真正的无线自由。",
        "价格": 99.99
    },
    "CineView 4K TV": {
        "名字": "CineView 4K TV",
        "分类": "电视和家庭影院系统",
        "品牌": "CineView",
        "型号": "CV-4K55",
        "保修": "2 年",
        "评分": 4.8,
        "特点": ["55英寸显示", "4K分辨率", "HDR", "智能电视"],
        "描述": "一台色彩鲜艳、功能智能的4K电视。",
        "价格": 599.99
    },
    "SoundMax Home Theater": {
        "名字": "SoundMax Home Theater",
        "分类": "电视和家庭影院系统",
        "品牌": "SoundMax",
        "型号": "SM-HT100",
        "保修": "1 年",
        "评分": 4.4,
        "特点": ["5.1声道", "1000W输出", "无线低音炮", "Bluetooth"],
        "描述": "一个为您提供身临其境音频体验的强大家庭影院系统。",
        "价格": 399.99
    },
    "CineView 8K TV": {
        "名字": "CineView 8K TV",
        "分类": "电视和家庭影院系统",
        "品牌": "CineView",
        "型号": "CV-8K65",
        "保修": "2 年",
        "评分": 4.9,
        "特点": ["65英寸显示", "8K分辨率", "HDR", "智能电视"],
        "描述": "用这款惊艳的8K电视体验电视的未来。",
        "价格": 2999.99
    },
    "SoundMax Soundbar": {
        "名字": "SoundMax Soundbar",
        "分类": "电视和家庭影院系统",
        "品牌": "SoundMax",
        "型号": "SM-SB50",
        "保修": "1 年",
        "评分": 4.3,
        "特点": ["2.1声道", "300W输出", "无线低音炮", "Bluetooth"],
        "描述": "用这款流线型且功能强大的音响条升级您的电视音频。",
        "价格": 199.99
    },
    "CineView OLED TV": {
        "名字": "CineView OLED TV",
        "分类": "电视和家庭影院系统",
        "品牌": "CineView",
        "型号": "CV-OLED55",
        "保修": "2 年",
        "评分": 4.7,
        "特点": ["55英寸显示", "4K分辨率", "HDR", "智能电视"],
        "描述": "用这款OLED电视体验真正的黑色和鲜艳的色彩。",
        "价格": 1499.99
    },
        "GameSphere X": {
        "名称": "GameSphere X",
        "分类": "游戏主机及配件",
        "品牌": "GameSphere",
        "型号": "GS-X",
        "保修": "1 年",
        "评分": 4.9,
        "特点": ["4K 游戏", "1TB 存储", "向后兼容", "在线多人游戏"],
        "描述": "下一代游戏主机，为您提供终极的游戏体验。",
        "价格": 499.99
    },
    "ProGamer Controller": {
        "名称": "ProGamer Controller",
        "分类": "游戏主机及配件",
        "品牌": "ProGamer",
        "型号": "PG-C100",
        "保修": "1 年",
        "评分": 4.2,
        "特点": ["人体工学设计", "可定制按键", "无线", "可充电电池"],
        "描述": "高品质的游戏控制器，为您提供精准和舒适的体验。",
        "价格": 59.99
    },
    "GameSphere Y": {
        "名称": "GameSphere Y",
        "分类": "游戏主机及配件",
        "品牌": "GameSphere",
        "型号": "GS-Y",
        "保修": "1 年",
        "评分": 4.8,
        "特点": ["4K 游戏", "500GB 存储", "向后兼容", "在线多人游戏"],
        "描述": "体积小巧的游戏主机，性能强大。",
        "价格": 399.99
    },
    "ProGamer Racing Wheel": {
        "名称": "ProGamer Racing Wheel",
        "分类": "游戏主机及配件",
        "品牌": "ProGamer",
        "型号": "PG-RW200",
        "保修": "1 年",
        "评分": 4.5,
        "特点": ["力反馈", "可调踏板", "拨片式换档", "与 GameSphere X 兼容"],
        "描述": "使用这款真实的赛车方向盘增强您的赛车游戏体验。",
        "价格": 249.99
    },
    "GameSphere VR Headset": {
        "名称": "GameSphere VR Headset",
        "分类": "游戏主机及配件",
        "品牌": "GameSphere",
        "型号": "GS-VR",
        "保修": "1 年",
        "评分": 4.6,
        "特点": ["身临其境的 VR 体验", "内置耳机", "可调节头带", "与 GameSphere X 兼容"],
        "描述": "带上这款舒适的 VR 头盔，步入虚拟现实的世界。",
        "价格": 299.99
    },
    "AudioPhonic Noise-Canceling Headphones": {
        "名称": "AudioPhonic Noise-Canceling Headphones",
        "分类": "音频设备",
        "品牌": "AudioPhonic",
        "型号": "AP-NC100",
        "保修": "1 年",
        "评分": 4.6,
        "特点": ["主动降噪", "蓝牙", "续航20小时", "舒适合身"],
        "描述": "使用这款降噪耳机体验沉浸式的声音。",
        "价格": 199.99
    },
        "WaveSound Bluetooth Speaker": {
        "名字": "WaveSound Bluetooth Speaker",
        "分类": "音响设备",
        "品牌": "WaveSound",
        "型号": "WS-BS50",
        "保修": "1年",
        "评分": 4.5,
        "功能": ["便携式", "10小时续航", "防水", "内置麦克风"],
        "描述": "一个紧凑而多功能的蓝牙音箱，适合随身携带。",
        "价格": 49.99
    },
    "AudioPhonic True Wireless Earbuds": {
        "名字": "AudioPhonic True Wireless Earbuds",
        "分类": "音响设备",
        "品牌": "AudioPhonic",
        "型号": "AP-TW20",
        "保修": "1年",
        "评分": 4.4,
        "功能": ["真正的无线", "蓝牙5.0", "触摸控制", "18小时续航"],
        "描述": "舒适的真无线耳塞，无需线缆即可享受音乐。",
        "价格": 79.99
    },
    "WaveSound Soundbar": {
        "名字": "WaveSound Soundbar",
        "分类": "音响设备",
        "品牌": "WaveSound",
        "型号": "WS-SB40",
        "保修": "1年",
        "评分": 4.3,
        "功能": ["2.0声道", "80W输出", "蓝牙", "可挂墙"],
        "描述": "使用这款纤薄且功能强大的音响条提升您电视的音质。",
        "价格": 99.99
    },
    "AudioPhonic Turntable": {
        "名字": "AudioPhonic Turntable",
        "分类": "音响设备",
        "品牌": "AudioPhonic",
        "型号": "AP-TT10",
        "保修": "1年",
        "评分": 4.2,
        "功能": ["3速", "内置扬声器", "蓝牙", "USB录音"],
        "描述": "使用这款现代唱机重新发现您的黑胶唱片收藏。",
        "价格": 149.99
    },
    "FotoSnap DSLR Camera": {
        "名字": "FotoSnap DSLR Camera",
        "分类": "摄像机",
        "品牌": "FotoSnap",
        "型号": "FS-DSLR200",
        "保修": "1年",
        "评分": 4.7,
        "功能": ["24.2MP传感器", "1080p视频", "3英寸LCD", "可更换镜头"],
        "描述": "使用这款多功能DSLR相机捕捉出色的照片和视频。",
        "价格": 599.99
    },
    "ActionCam 4K": {
        "名字": "ActionCam 4K",
        "分类": "摄像机",
        "品牌": "ActionCam",
        "型号": "AC-4K",
        "保修": "1年",
        "评分": 4.4,
        "功能": ["4K视频", "防水", "图像稳定", "Wi-Fi"],
        "描述": "使用这款坚固而紧凑的4K动作摄像机记录您的冒险。",
        "价格": 299.99
    },
    "FotoSnap Mirrorless Camera": {
        "名字": "FotoSnap Mirrorless Camera",
        "分类": "摄像机",
        "品牌": "FotoSnap",
        "型号": "FS-ML100",
        "保修": "1年",
        "评分": 4.6,
        "功能": ["20.1MP传感器", "4K视频", "3英寸触摸屏", "可更换镜头"],
        "描述": "一款轻巧且功能先进的无反光镜相机。",
        "价格": 799.99
    },
    "ZoomMaster Camcorder": {
        "名字": "ZoomMaster Camcorder",
        "分类": "摄像机",
        "品牌": "ZoomMaster",
        "型号": "ZM-CM50",
        "保修": "1年",
        "评分": 4.3,
        "功能": ["1080p视频", "30倍光学变焦", "3英寸LCD", "图像稳定"],
        "描述": "使用这款易于使用的摄像机捕捉生活中的时刻。",
        "价格": 249.99
    },
    "FotoSnap Instant Camera": {
        "名字": "FotoSnap Instant Camera",
        "分类": "摄像机",
        "品牌": "FotoSnap",
        "型号": "FS-IC10",
        "保修": "1年",
        "评分": 4.1,
        "功能": ["即时打印", "内置闪光灯", "自拍镜", "电池供电"],
        "描述": "使用这款有趣且便携的即时相机创建即时记忆。",
        "价格": 69.99
    },
}


In [59]:
def get_product_by_name(name):
    return products.get(name, None)

def get_products_by_category(category):
    return [product for product in products.values() if product["分类"] == category]

In [60]:
print(get_product_by_name("TechPro Ultrabook"))

{'名称': 'TechPro Ultrabook', '分类': '电脑与笔记本', '品牌': 'TechPro', '型号': 'TP-UB100', '保修期': '1 年', '评分': 4.5, '特点': ['13.3 英寸显示屏', '8GB RAM', '256GB SSD', 'Intel Core i5 处理器'], '描述': '轻薄的日常使用超级本。', '价格': 799.99}


In [61]:
print(get_products_by_category("摄像机"))

[{'名字': 'FotoSnap DSLR Camera', '分类': '摄像机', '品牌': 'FotoSnap', '型号': 'FS-DSLR200', '保修': '1年', '评分': 4.7, '功能': ['24.2MP传感器', '1080p视频', '3英寸LCD', '可更换镜头'], '描述': '使用这款多功能DSLR相机捕捉出色的照片和视频。', '价格': 599.99}, {'名字': 'ActionCam 4K', '分类': '摄像机', '品牌': 'ActionCam', '型号': 'AC-4K', '保修': '1年', '评分': 4.4, '功能': ['4K视频', '防水', '图像稳定', 'Wi-Fi'], '描述': '使用这款坚固而紧凑的4K动作摄像机记录您的冒险。', '价格': 299.99}, {'名字': 'FotoSnap Mirrorless Camera', '分类': '摄像机', '品牌': 'FotoSnap', '型号': 'FS-ML100', '保修': '1年', '评分': 4.6, '功能': ['20.1MP传感器', '4K视频', '3英寸触摸屏', '可更换镜头'], '描述': '一款轻巧且功能先进的无反光镜相机。', '价格': 799.99}, {'名字': 'ZoomMaster Camcorder', '分类': '摄像机', '品牌': 'ZoomMaster', '型号': 'ZM-CM50', '保修': '1年', '评分': 4.3, '功能': ['1080p视频', '30倍光学变焦', '3英寸LCD', '图像稳定'], '描述': '使用这款易于使用的摄像机捕捉生活中的时刻。', '价格': 249.99}, {'名字': 'FotoSnap Instant Camera', '分类': '摄像机', '品牌': 'FotoSnap', '型号': 'FS-IC10', '保修': '1年', '评分': 4.1, '功能': ['即时打印', '内置闪光灯', '自拍镜', '电池供电'], '描述': '使用这款有趣且便携的即时相机创建即时记忆。', '价格': 69.99}]


In [62]:
import json 

def read_string_to_list(input_string):
    if input_string is None:
        return None

    try:
        input_string = input_string.replace("'", "\"")  # Replace single quotes with double quotes for valid JSON
        data = json.loads(input_string)
        return data
    except json.JSONDecodeError:
        print("Error: Invalid JSON string")
        return None   

In [63]:
category_and_product_list = read_string_to_list(category_and_product_response_1)
print(category_and_product_list)

[{'category': '手机和配件'}, {'products': ['SmartX ProPhone']}, {'category': '摄像机和摄像机'}, {'products': ['FotoSnap DSLR Camera']}, {'category': '电视和家庭影院系统'}]


In [68]:
def generate_output_string(data_list):
    output_string = ""

    if data_list is None:
        return output_string

    for data in data_list:
        try:
            if "products" in data:
                products_list = data["products"]
                for product_name in products_list:
                    product = get_product_by_name(product_name)
                    if product:
                        output_string += json.dumps(product, indent=4, ensure_ascii=False) + "\n"
                    else:
                        print(f"Error: Product '{product_name}' not found")
            elif "category" in data:
                category_name = data["category"]
                category_products = get_products_by_category(category_name)
                for product in category_products:
                    output_string += json.dumps(product, indent=4, ensure_ascii=False) + "\n"
            else:
                print("Error: Invalid object format")
        except Exception as e:
            print(f"Error: {e}")

    return output_string 

In [69]:
product_information_for_user_message_1 = generate_output_string(category_and_product_list)
print(product_information_for_user_message_1)

{
    "名称": "SmartX ProPhone",
    "分类": "智能手机和配件",
    "品牌": "SmartX",
    "型号": "SX-PP10",
    "保修期": "1年",
    "评分": 4.6,
    "特点": [
        "6.1英寸显示屏",
        "128GB 存储",
        "12MP 双摄像头",
        "5G"
    ],
    "描述": "具有先进摄像头功能的强大智能手机。",
    "价格": 899.99
}
{
    "名字": "FotoSnap DSLR Camera",
    "分类": "摄像机",
    "品牌": "FotoSnap",
    "型号": "FS-DSLR200",
    "保修": "1年",
    "评分": 4.7,
    "功能": [
        "24.2MP传感器",
        "1080p视频",
        "3英寸LCD",
        "可更换镜头"
    ],
    "描述": "使用这款多功能DSLR相机捕捉出色的照片和视频。",
    "价格": 599.99
}
{
    "名字": "CineView 4K TV",
    "分类": "电视和家庭影院系统",
    "品牌": "CineView",
    "型号": "CV-4K55",
    "保修": "2 年",
    "评分": 4.8,
    "特点": [
        "55英寸显示",
        "4K分辨率",
        "HDR",
        "智能电视"
    ],
    "描述": "一台色彩鲜艳、功能智能的4K电视。",
    "价格": 599.99
}
{
    "名字": "SoundMax Home Theater",
    "分类": "电视和家庭影院系统",
    "品牌": "SoundMax",
    "型号": "SM-HT100",
    "保修": "1 年",
    "评分": 4.4,
    "特点": [
        "5.1声道",
        "1000W输出",
      

In [70]:
system_message = f"""
您是一家大型电子商店的客户服务助理。\
请以友好和有帮助的语气回复，\
确保答案简明扼要。\
确保询问用户相关的后续问题。
"""
user_message_1 = f"""
告诉我关于smartx pro手机和\
fotosnap相机，那个DSLR的。\
还有告诉我关于你们的电视"""
messages =  [  
{'role':'system',
 'content': system_message},   
{'role':'user',
 'content': user_message_1},  
{'role':'assistant',
 'content': f"""相关产品信息:\n\
 {product_information_for_user_message_1}"""},   
]
final_response = get_completion_from_messages(messages)
print(final_response)


以上是我们店里的一些产品信息，SmartX Pro手机是一款功能强大的智能手机，具有先进的摄像头功能和5G网络。FotoSnap DSLR相机是一款多功能的相机，具有24.2MP传感器和可更换镜头。我们还有多款电视和家庭影院系统，包括CineView 4K电视、CineView 8K电视和CineView OLED电视，以及SoundMax家庭影院系统和音响条。如果您有任何关于这些产品的问题或需要更多信息，请告诉我，我会尽力帮助您。


In [92]:

def process_user_message(user_input, all_messages, debug=True):
    delimiter = "```"
    
    category_and_product_response = find_category_and_product_only(user_input)
    print(category_and_product_response)
    # Step 1: Extract the list of products
    category_and_product_list = read_string_to_list(category_and_product_response)
    print(category_and_product_list)

    if debug: print("Step 2: Extracted list of products.")

    # Step 2: If products are found, look them up
    product_information = generate_output_string(category_and_product_list)
    if debug: print("Step 3: Looked up product information.")

    # Step 3: Answer the user question
    system_message = f"""
        您是一家大型电子商店的客户服务助理。\
        请以友好和有帮助的语气回复，\
        确保答案简明扼要。\
        确保询问用户相关的后续问题。
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': f"{delimiter}{user_input}{delimiter}"},
        {'role': 'assistant', 'content': f"Relevant product information:\n{product_information}"}
    ]

    final_response = get_completion_from_messages(all_messages + messages)
    if debug:print("Step 4: Generated response to user question.")
    all_messages = all_messages + messages[1:]

    # Step 4: Ask the model if the response answers the initial user query well
    user_message = f"""
    Customer message: {delimiter}{user_input}{delimiter}
    Agent response: {delimiter}{final_response}{delimiter}

    Does the response sufficiently answer the question?
    """
    messages = [
        {'role': 'system', 'content': system_message},
        {'role': 'user', 'content': user_message}
    ]
    evaluation_response = get_completion_from_messages(messages)

    # Step 5: If yes, use this answer; if not, say that you will connect the user to a human
    if "Y" in evaluation_response:  # Using "in" instead of "==" to be safer for model output variation (e.g., "Y." or "Yes")
        if debug: print("Step 7: Model approved the response.")
        return final_response, all_messages
    else:
        if debug: print("Step 7: Model disapproved the response.")
        neg_str = "I'm unable to provide the information you're looking for. I'll connect you with a human representative for further assistance."
        return neg_str, all_messages

user_input = "告诉我关于smartx pro手机和fotosnap相机，那个DSLR的。还有，告诉我关于你们的电视。"
response,_ = process_user_message(user_input,[])
print(response)

[{'category': '手机和配件'}, {'products': ['SmartX ProPhone', 'FotoSnap DSLR Camera']}, {'category': '电视和家庭影院系统'}]
[{'category': '手机和配件'}, {'products': ['SmartX ProPhone', 'FotoSnap DSLR Camera']}, {'category': '电视和家庭影院系统'}]
Step 2: Extracted list of products.
Step 3: Looked up product information.
Step 4: Generated response to user question.
Step 7: Model approved the response.
关于SmartX Pro手机和FotoSnap相机，SmartX Pro手机是一款功能强大的智能手机，具有6.1英寸显示屏，128GB存储，12MP双摄像头和5G等特点。FotoSnap相机是一款多功能DSLR相机，具有24.2MP传感器，1080p视频，3英寸LCD和可更换镜头等特点。如果您需要更多信息，请告诉我您感兴趣的方面。

关于我们的电视，我们有多款电视可供选择，包括CineView 4K电视、CineView 8K电视和CineView OLED电视等。这些电视都具有不同的特点和价格，以满足不同客户的需求。如果您需要更多信息，请告诉我您感兴趣的方面。


In [94]:

import logging

logging.basicConfig(level=logging.DEBUG, filename='debug.log', filemode='w')
logging.debug("Script started.")

import panel as pn  # GUI
pn.extension()
def collect_messages(debug=True):
    logging.debug("collect_messages function has been called.")
    logging.shutdown()

    user_input = inp.value_input
    logging.debug(f"User Input = {user_input}")
        
    if user_input == "":
        return
    inp.value = ''
    global context
    #response, context = process_user_message(user_input, context, utils.get_products_and_category(),debug=True)
    response, context = process_user_message(user_input, context, debug=debug)
    context.append({'role':'assistant', 'content':f"{response}"})
    panels.append(
        pn.Row('User:', pn.pane.Markdown(user_input, width=600)))
    panels.append(
        pn.Row('Assistant:', pn.pane.Markdown(response, width=600, style={'background-color': '#F6F6F6'})))
 
    return pn.Column(*panels)


panels = [] # collect display 

context = [ {'role':'system', 'content':"You are Service Assistant"} ]  

inp = pn.widgets.TextInput( placeholder='Enter text here…')
button_conversation = pn.widgets.Button(name="Service Assistant")

interactive_conversation = pn.bind(collect_messages, button_conversation)

dashboard = pn.Column(
    inp,
    pn.Row(button_conversation),
    pn.panel(interactive_conversation, loading_indicator=True, height=300),
)

dashboard

Column
    [0] TextInput(placeholder='Enter text here…')
    [1] Row
        [0] Button(name='Service Assistant')
    [2] ParamFunction(function, _pane=Str, defer_load=False, height=300, loading_indicator=True)